In [1]:
# Import Libraries
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np

# Paths
MESH_FILE = r"c:\projetos\projeto-censo\data\spatial\malha_setores_2022_final.parquet"
DIAMOND_FILE = r"c:\projetos\projeto-censo\data\diamond\censo_2022_diamond_features_final.parquet"

# Load Data
print("Loading Mesh (Registry)...")
gdf_mesh = gpd.read_parquet(MESH_FILE)
print(f"Mesh Shape: {gdf_mesh.shape}")

print("Loading Diamond Features...")
df_features = pd.read_parquet(DIAMOND_FILE)
print(f"Features Shape: {df_features.shape}")

# Merge
print("Merging Mesh and Features...")
gdf_full = gdf_mesh.merge(df_features, on='id_setor', how='inner')
print(f"Merged Shape: {gdf_full.shape}")

# Filter for Target Districts (Case Insensitive)
target_districts = ['SANTANA', 'CASA VERDE', 'BARRA FUNDA']
gdf_full['nm_distrito_upper'] = gdf_full['nm_distrito'].str.upper()
gdf_study = gdf_full[gdf_full['nm_distrito_upper'].isin(target_districts)].copy()

print(f"Study Area Shape: {gdf_study.shape}")
print("Districts found:", gdf_study['nm_distrito'].unique())

Loading Mesh (Registry)...
Mesh Shape: (468097, 15)
Loading Diamond Features...
Features Shape: (458772, 1097)
Merging Mesh and Features...
Merged Shape: (458772, 1111)
Study Area Shape: (851, 1112)
Districts found: ['Santana' 'Barra Funda' 'Casa Verde']


In [2]:
# Geocode Address
address = "Rua Marechal Hermes da Fonseca, 722 - Santana - São Paulo"
geolocator = Nominatim(user_agent="census_study")
location = geolocator.geocode(address)

if location:
    user_lat = location.latitude
    user_lon = location.longitude
    print(f"Address found: {location.address}")
    print(f"Coordinates: {user_lat}, {user_lon}")
else:
    print("Address not found. Using approximate coordinates for Santana.")
    user_lat = -23.502
    user_lon = -46.625

# Create Point Geometry
from shapely.geometry import Point
user_point = Point(user_lon, user_lat)

# Find the census tract containing the point
# Ensure CRS match. Mesh is likely EPSG:4674 (SIRGAS 2000) or 4326.
# Let's check CRS
print(f"Mesh CRS: {gdf_study.crs}")

# Create a GeoDataFrame for the user point
gdf_user = gpd.GeoDataFrame({'geometry': [user_point]}, crs="EPSG:4326")

# If Mesh is not 4326, reproject user point
if gdf_study.crs != "EPSG:4326":
    gdf_user = gdf_user.to_crs(gdf_study.crs)

# Spatial Join to find the tract
user_tract = gpd.sjoin(gdf_user, gdf_study, how="left", predicate="within")

if not user_tract.empty:
    tract_id = user_tract.iloc[0]['id_setor']
    print(f"User is in Census Tract: {tract_id}")
    print(f"District: {user_tract.iloc[0]['nm_distrito']}")
    print(f"Neighborhood: {user_tract.iloc[0]['nm_bairro']}")
else:
    print("User point falls outside the study area tracts!")

Address found: Edifício Solar das Laranjeiras, 722, Rua Marechal Hermes da Fonseca, Santa Teresinha, Alto de Santana, Santana, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 02020-001, Brasil
Coordinates: -23.4939815, -46.6327027
Mesh CRS: {"$schema": "https://proj.org/schemas/v0.7/projjson.schema.json", "type": "GeographicCRS", "name": "SIRGAS 2000", "datum": {"type": "GeodeticReferenceFrame", "name": "Sistema de Referencia Geocentrico para las AmericaS 2000", "ellipsoid": {"name": "GRS 1980", "semi_major_axis": 6378137, "inverse_flattening": 298.257222101}}, "coordinate_system": {"subtype": "ellipsoidal", "axis": [{"name": "Geodetic latitude", "abbreviation": "Lat", "direction": "north", "unit": "degree"}, {"name": "Geodetic longitude", "abbreviation": "Lon", "direction": "east", "unit": "degree"}]}, "scope": "Horizontal component of 3D system.", "area": "Latin America - Central Amer

In [4]:
# Visualize Region and Risk Profile

# 1. Map of the Region with Multiple Layers
# Create a base map centered on the user
m = folium.Map(location=[user_lat, user_lon], zoom_start=14, tiles="cartodbpositron")

# Add User Marker
folium.Marker(
    [user_lat, user_lon],
    popup="My Location",
    icon=folium.Icon(color="red", icon="home")
).add_to(m)

# Define Layers to Plot
layers_to_plot = {
    "Renda Média (SM)": "rendimento_medio_responsavel_sm",
    "Potencial Consumo (Index)": "expert_econ_consumption_potential_index",
    "Diversidade (Index)": "expert_social_racial_diversity_index",
    "Envelhecimento (Index)": "expert_demog_aging_index",
    "LISA Consumo (Cluster)": "expert_econ_consumption_potential_index_lisa_q_k5",
    "LISA Diversidade (Cluster)": "expert_social_racial_diversity_index_lisa_q_k5"
}

# Add Layers
for label, col in layers_to_plot.items():
    if col in gdf_study.columns:
        # Handle NaN for plotting
        plot_data = gdf_study.dropna(subset=[col])
        
        # Determine colorscale based on type (LISA is categorical 1-4, others continuous)
        if 'lisa_q' in col:
            fill_color = 'Set1' # Categorical-ish
            bins = 4
        else:
            fill_color = 'YlOrRd'
            bins = 6
            
        folium.Choropleth(
            geo_data=plot_data,
            name=label,
            data=plot_data,
            columns=['id_setor', col],
            key_on='feature.properties.id_setor',
            fill_color=fill_color,
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name=label,
            highlight=True,
            show=(label == "Renda Média (SM)") # Only show first by default
        ).add_to(m)

# Add Layer Control to switch between indicators
folium.LayerControl().add_to(m)

# Save Map
m.save("santana_multilayer_map.html")
print("Map saved to santana_multilayer_map.html")

# 2. Expanded Radar Chart
if not user_tract.empty:
    user_row = user_tract.iloc[0]
    
    # Select Features for Radar Chart
    radar_features = {
        "Renda (SM)": "rendimento_medio_responsavel_sm",
        "Potencial Consumo": "expert_econ_consumption_potential_index",
        "Diversidade": "expert_social_racial_diversity_index",
        "Envelhecimento": "expert_demog_aging_index",
        "Isolamento Econ": "expert_econ_consumption_potential_index_isolation_k5",
        "Desigualdade Econ": "expert_econ_consumption_potential_index_inequality_k5"
    }
    
    # Filter available columns
    available_radar = {k: v for k, v in radar_features.items() if v in gdf_study.columns}
    
    if available_radar:
        # Normalize data for Radar Chart (0-1 scale relative to district max) to make them comparable
        # Or just plot raw values if scales are similar? 
        # Indices are usually 0-1 or standardized. Income is in SM (1-20+). Isolation is 0-10+.
        # Better to normalize against the District Max for visualization
        
        categories = list(available_radar.keys())
        user_values = []
        district_values = []
        
        for label, col in available_radar.items():
            max_val = gdf_study[col].max()
            if max_val == 0: max_val = 1
            
            u_val = user_row[col] / max_val
            d_val = gdf_study[gdf_study['nm_distrito'] == user_row['nm_distrito']][col].mean() / max_val
            
            user_values.append(u_val)
            district_values.append(d_val)
            
        fig = go.Figure()
        
        fig.add_trace(go.Scatterpolar(
            r=user_values,
            theta=categories,
            fill='toself',
            name='My Location (Normalized)'
        ))
        fig.add_trace(go.Scatterpolar(
            r=district_values,
            theta=categories,
            fill='toself',
            name=f"Avg {user_row['nm_distrito']} (Normalized)"
        ))
        
        fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, 1]
                )),
            showlegend=True,
            title="Relative Profile (Normalized to District Max)"
        )
        
        fig.show()
        fig.write_html("santana_expanded_profile.html")
        print("Expanded profile chart saved to santana_expanded_profile.html")
        
        # Print Raw Values for context
        print("\n--- Raw Values ---")
        for label, col in available_radar.items():
            print(f"{label}: {user_row[col]:.4f} (Avg: {gdf_study[gdf_study['nm_distrito'] == user_row['nm_distrito']][col].mean():.4f})")
            
    else:
        print("No radar features found.")

Map saved to santana_multilayer_map.html


Expanded profile chart saved to santana_expanded_profile.html

--- Raw Values ---
Renda (SM): 7.2803 (Avg: 4.4039)
Potencial Consumo: 30.3034 (Avg: 16.3307)
Diversidade: 0.3208 (Avg: 0.3823)
Envelhecimento: 0.9808 (Avg: 0.6777)
Isolamento Econ: 0.8033 (Avg: 4.3147)
Desigualdade Econ: 0.5179 (Avg: 0.3283)
